In [1]:
import numpy as np
from collections import defaultdict, Counter
from faker import Faker
from tabulate import tabulate

fake = Faker()

## Draw Simulation Methods

In [2]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = defaultdict(lambda: 'Bye'), defaultdict(lambda: 1600)
    for n in range(num_players):
        p_id, p_name, p_rank = str(np.random.randint(1000000, 9999999)), fake.name(), np.random.randint(1600, 2000)
        players.append([p_id, p_name, p_rank])
        id_to_name[p_id] = p_name
        id_to_rank[p_id] = p_rank
    return players, id_to_name, id_to_rank

In [3]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [4]:
def run_round(pairings, rnd, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = 0
    for pairing in pairings:
        a, b, pod = pairing[0][1], pairing[1][1], pairing[2]
        if a[:3] == 'BYE':
            result = [a,b,b]
        elif b[:3] == 'BYE':
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result + [rnd, pod])
        
    return results

## Draw Calculation Methods

In [5]:
class Points():
    """
    Hyperparameter set for scoring pairings
    """
    valid                          =  16
    self_vs_self                   = -2048
    not_in_pod                     = -2048
    repeat_opponent_within_format  = -512
    repeat_opponent_between_format = -2
    multiple_byes                  = -2
    bye_last_round                 = -512
    overall_win_distance           = -8
    tiebreak_distance              = -4

BYE  = 0
PID  = 0
WINS = 1
TBRK = 2
POD  = 3
PODW = 4

def create_draw(players, results, rounds, mode='production', pod_size=8, minimiseTiebreakDist=False):
    """
    Create draw for the round

    Parameters
    ----------
    players: Array of player details
        (n x 3) list containing [player_id, player_name, player_ranking]
    results: List of pairings and results for all rounds to date
        (m x 5) list containing [player_a_id, player_b_id, winner_id, round_id, pod_id]
    rounds: List of round types
        (n x 0) list of {'swissx' | 'podx'}
    mode: string
        System run mode. Returns reduced form outputs in production mode
    pod_size: int
        size of pods for the system to create if round is the start of a new pod
    minimiseTiebreakDist: Flag if final round. Pair scores will include distance between tiebreakers
        boolean

    Returns
    -------
    standings: List of standings for all players still in the tournament
        (n x 2) list containing [player_id, player_wins]
    splits: List of index points where each points bracket finishes
    scores: Matrix containing the scores for all possible pairings
        (n x n) matrix of floats
    alloc: The matrix form pairing allocations
        (n x n) matrix of integers
    pairs: The list form of pairing allocations
        (n / 2 x (2, 2, 1)) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id), pod]
    """
    n_players = len(players)

    standings               = get_standings(players, results, rounds, pod_size)
    scores                  = get_scores(standings, results, rounds, minimiseTiebreakDist)
    alloc, pairs, splits    = get_allocations(standings, scores, mode)

    return standings, splits, scores, alloc, pairs

def get_standings(players, results, rounds, pod_size=8, this_round=None, minimiseTiebreakDist=False):
    """
    Calculate player standings at start of round

    Parameters
    ----------
    players: Array of player details
        (n x 3) list containing [player_id, player_name, player_ranking]
    results: List of pairings and results for all rounds to date
        (m x 5) list containing [player_a_id, player_b_id, winner_id, round_id, pod_id]
    rounds: List of round types
        (n x 0) list of {'swissx' | 'podx'}
    pod_size: int
        Number of players per pod
    this_round: int
        Set the current round that standings are being calculated for
        If "None" then it will calculate for the maximum round contained in "results" plus 1

    Returns
    -------
    standings: List of standings for all players still in the tournament
        (n x 3) list containing [player_id, player_wins, player_pod]
    """
    if this_round == None:
        this_round  = 0 if len(results) == 0 else np.array(results)[:,3].astype(int).max() + 1
    last_round        = this_round - 1
    win_history       = defaultdict(lambda: np.zeros(max(1, this_round), dtype=int))
    pod_standings     = defaultdict(int)
    pods              = defaultdict(lambda: 1)
    active_players    = set([player[0] for player in players])

    # Use all rounds for pod standing if 'swiss', or just pod results if 'pod'
    same_format = set([n for n, rnd in enumerate(rounds) if rnd == rounds[this_round]])

    # Calculate player standings
    if len(results) > 0:
        for a, b, winner, rnd, pod in results:
            win_history[a][rnd] += (a == winner)
            win_history[b][rnd] += (b == winner)
            if rnd in same_format:
                pod_standings[a] += (a == winner)
                pod_standings[b] += (b == winner)
            if rnd == last_round:
                if a in active_players: pods[a] = pod
                if b in active_players: pods[b] = pod

    # Generate weightings for tiebreakers
    w = 0.25 ** np.arange(max(1, this_round))
    w = w.cumsum()[::-1]

    standings = np.zeros((len(active_players), 5), dtype=object)
    for n, p in enumerate(active_players):
        standings[n,PID]  = p
        standings[n,WINS] = win_history[p].sum()
        standings[n,TBRK] = (w*win_history[p]).sum() / w.sum()

    standings = sort(standings, [WINS], [-1], preshuffle=True)

    # Determine whether to create, reuse or ignore pods
    if rounds[this_round][:3] == 'pod':
        if this_round == 0 or rounds[this_round] != rounds[last_round]:
            method = 'create pod'
        else:
            method = 'reuse pod'
    else:
        method = 'bracket'

    # Allocate players to pods
    for pos, standing in enumerate(standings):
        player_id = standing[PID]
        if method == 'create pod':
            if pos < (len(players) // pod_size) * pod_size:
                standing[POD] = pos // pod_size + 1
            elif (len(players) % pod_size) / pod_size < 0.5:
                standing[POD] = pos // pod_size
            else:
                standing[POD] = pos // pod_size + 1
        elif method == 'reuse pod':
            standing[POD] = pods[player_id]
        else:
            standing[POD] = 1

    # Add byes into the largest bracket in each of the pods
    pod_sizes = Counter(standings[:,POD])
    byes = []
    for pod_num in range(1, len(pod_sizes) + 1):
        if pod_sizes[pod_num] % 2 == 1:
            min_overall_ranking  = standings[standings[:,POD]==pod_num][:,WINS].min()
            min_pod_ranking      = standings[standings[:,POD]==pod_num][:,PODW].min()
            min_pod_tiebreak     = standings[standings[:,POD]==pod_num][:,TBRK].min()
            byes.append(['BYE' + str(pod_num), min_overall_ranking, min_pod_tiebreak, pod_num, min_pod_ranking])

    if len(byes) > 0:
        byes = np.array(byes, dtype=object)
        standings = np.vstack([standings, byes])

    standings = sort(standings, [TBRK, WINS, POD], [1, -1, 1], preshuffle=True)

    return standings

def get_scores(standings, results, rounds, minimiseTiebreakDist=False):
    """
    Assign scores to each possible pairing
    A higher score means that the pairing is more strongly preferred

    Parameters
    ----------
    standings: List of standings for all players still in the tournament
        (n x 2) list containing [player_id, player_wins]
    results: List of pairings and results for all rounds to date
        (m x 5) list containing [player_a_id, player_b_id, winner_id, round_id, pod_id]
    rounds: List of round types
        (n x 0) list of {'swissx' | 'podx'}
    minimiseTiebreakDist: Flag if final round. Pair scores will include distance between tiebreakers
        boolean

    Returns
    -------
    scores: Matrix containing the scores for all possible pairings
        (n x n) matrix of floats
    """
    # Get player data
    n_players = len(standings)
    ix = {player_id: pos for pos, player_id in enumerate(standings[:,PID])}
    byes_ix = [n for n, standing in enumerate(standings[:,PID]) if standing[:3] == 'BYE']
    active_players = set(ix.keys())

    # Get round data
    this_round  = 0 if len(results) == 0 else np.array(results)[:,3].astype(int).max() + 1
    same_format = set([n for n, rnd in enumerate(rounds) if rnd == rounds[this_round]])

    # Calculate scores for pairings
    scores = np.full((n_players, n_players), Points.valid)

    # Set self vs self to -inf
    np.fill_diagonal(scores, Points.self_vs_self)

    # Set pairs in different pod to -inf
    not_in_pod = standings[:,POD:POD+1] != standings[:,POD:POD+1].T
    scores += Points.not_in_pod * not_in_pod

    def apply_penalty(scores, ix_player_a, ix_player_b, penalty):
        scores[ix_player_a, ix_player_b] += penalty
        scores[ix_player_b, ix_player_a] += penalty

    # Subtract points for prior matchup
    for player_a, player_b, winner, rnd, pod_id in results:
        if player_a in active_players and player_b in active_players:
            if   player_a[:3] != 'BYE' and player_b[:3] != 'BYE' and rnd in same_format:
                apply_penalty(scores, ix[player_a], ix[player_b], Points.repeat_opponent_within_format)
            elif player_a[:3] != 'BYE' and player_b[:3] != 'BYE' and rnd not in same_format and not minimiseTiebreakDist:
                apply_penalty(scores, ix[player_a], ix[player_b], Points.repeat_opponent_between_format)

    # Subtract points for repeat bye
    for player_a, player_b, winner, rnd, pod_id in results:
        penalty = Points.bye_last_round if rnd == this_round - 1 else Points.multiple_byes
        if player_a in active_players and player_b[:3] == 'BYE':
            apply_penalty(scores, ix[player_a], byes_ix, penalty)
        if player_b in active_players and player_a[:3] == 'BYE':
            apply_penalty(scores, byes_ix, ix[player_b], penalty)

    # Subtract points for different number of overall wins
    overall_win_distance = (standings[:,[WINS]] - standings[:,[WINS]].T) ** 2
    scores = scores + Points.overall_win_distance * overall_win_distance

    # Subtract points for different number of in-format wins
    if minimiseTiebreakDist:
        tiebreak_distance = 1 + 1e-12 - (np.abs(standings[:,[TBRK]] - standings[:,[TBRK]].T).astype(np.float32))
        scores += Points.tiebreak_distance  * -np.log(tiebreak_distance) * 25

    return scores

def get_allocations(standings, scores, mode):
    """
    Split table down into even length brackets and sub-allocate players to pairs within the bracket

    Parameters
    ----------
    standings: List of standings for all players still in the tournament
        (n x 4) list containing [player_id, player_wins, pod_id, player_wins_in_pod]
    scores: Matrix containing the scores for all possible pairings
        (n x n) matrix of floats

    Returns
    -------
    alloc: The matrix form pairing allocations
        (n x n) matrix of integers
    pairs: The list form of pairing allocations
        (n / 2 x (2, 2, 1)) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id), pod]
    """
    n_players = len(standings)
    splits = get_splits(standings)

    brackets = [n for n in range(n_players) if (standings[n,[WINS]] != standings[n-1,[WINS]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = Points.valid  - 2 * n_odd_brackets * (Points.valid + Points.overall_win_distance) / n_players

    # Run Gibbs Sampler, and return if it hit max result
    allocs_gs, pairs_gs = _gibbs_sampler(standings, scores)
    if scores[allocs_gs==1].mean() == max_score:
        return allocs_gs, pairs_gs, splits

    # Otherwise run Metropolis-Hastings Sampler
    # Allocate entire table if there is no split
    if len(splits) == 1:
        allocs, pairs = _get_allocations(standings, scores)

    else:
        allocs = np.zeros((n_players, n_players))
        pairs = []

        # Split into sub-tables at even numbered index split points
        ixs = np.array(splits)
        ixs = ixs[ixs % 2 == 0]
        for ix1, ix2 in zip(ixs[:-1], ixs[1:]):
            alloc, pair = _metroplis_hastings(standings[ix1:ix2], scores[ix1:ix2,ix1:ix2], start=ix1)
            allocs[ix1:ix2,ix1:ix2] = alloc
            pairs += pair

    allocs_mh, pairs_mh = _gibbs_sampler(standings, scores, alloc=allocs)

    # Return the better of Gibbs or Metropolis Hastings
    if(scores[allocs_mh==1].mean() >= scores[allocs_gs==1].mean()):
        return allocs_mh, pairs_mh, splits
    else:
        return allocs_gs, pairs_gs, splits

def _metroplis_hastings(standings, scores, start=0):
    """
    Allocate player pairings using MH probability method

    Parameters
    ----------
    standings: List of standings for all players still in the tournament
        (n x 4) list containing [player_id, player_wins, pod_id, player_wins_in_pod]
    scores: Matrix containing the scores for all possible pairings
        (n x n) matrix of floats

    Returns
    -------
    alloc: The matrix form pairing allocations
        (n x n) matrix of integers
    pairs: The list form of pairing allocations
        (n / 2 x (2, 2, 1)) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id), pod]
    """
    n_players = len(standings)
    ix = {pos+start: player_id for pos, player_id in enumerate(standings[:,PID])}
    alloc = np.zeros((n_players, n_players), dtype=int)
    pods = {player[PID]: player[POD] for player in standings}

    # Construct initial draw
    constraints = scores.astype(int) == scores.astype(int).max(axis=1, keepdims=True)
    constraints = n_players - constraints.sum(axis=1)
    prob = scores - scores.max(axis=1, keepdims=True)
    prob = np.exp(prob.astype(np.float32)/4)

    for _ in range(int(n_players / 2)):
        mask = (1 - alloc.sum(axis=1))
        # Choose Player A
        a = (constraints * mask).argmax()
        # Choose Player B
        p = np.maximum(prob[a] * mask, 0)
        b = np.random.choice(np.arange(n_players)[p == p.max()])
        #b = np.random.choice(range(n_players),p=p/p.sum())
        #print(a, '(', standings[a][1], ')', b, '(', standings[b][1], ')')

        alloc[a, b] = 1
        alloc[b, a] = 1

    brackets = [n for n in range(n_players) if (standings[n,[WINS]] != standings[n-1,[WINS]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = Points.valid  - 2 * n_odd_brackets * (Points.valid + Points.overall_win_distance) / n_players

    # Run Metropolis-Hastings to iteratively improve draw
    score_q = [(alloc*scores).sum()/len(scores)]
    if score_q[0] != max_score:
        prob /= prob.sum(axis=1, keepdims=True)

        best_alloc = alloc.copy()
        while len(score_q) < 100 or np.mean(score_q[-10:]) > np.mean(score_q[-20:-10]):
            #print('---', len(score_q), '---')
            if max(score_q) == max_score:
                break
            if len(score_q) > 10 and min(score_q[-10:]) == max(score_q[-10:]):
                break
            for a, p in enumerate(prob):
                b = alloc[a].argmax()
                c = np.random.choice(range(len(p)),p=p)
                d = alloc[c].argmax()
                p_change = prob[a,c]*prob[b,d]/(prob[a,b]*prob[c,d] + prob[a,c]*prob[b,d])
                if np.random.random() < p_change:
                    alloc = swap_pairs((a,b), (c,d), alloc)
                #    print(a, '(', standings[a][1], ')', b, '(', standings[b][1], ')')
                #else:
                #    print(a, '(', standings[a][1], ')', c, '(', standings[c][1], ')')

            score = (alloc*scores).sum()/len(prob)
            if score > max(score_q):
                best_alloc = alloc.copy()
            score_q.append(score)

        alloc = best_alloc
        del(best_alloc)

    #if max_score > max(score_q): print("***", len(score_q), max_score, max(score_q), "***")
    #if len(score_q) > 1: print(["%.2f" % v for v in score_q])
    # Create list form of pairings
    alloc_ = alloc * np.tri(*alloc.shape)

    pairs = []
    for n in range(n_players):
        if alloc_[n].max() == 1:
            a, b = n + start, alloc_[n].argmax() + start
            pairs.append([(a, ix[a]), (b, ix[b]), pods[ix[a]]])

    return alloc, pairs

def _gibbs_sampler(standings, scores, start=0, alloc=None):
    """
    Allocate player pairings using Gibbs' sampling method

    Parameters
    ----------
    standings: List of standings for all players still in the tournament
        (n x 4) list containing [player_id, player_wins, pod_id, player_wins_in_pod]
    scores: Matrix containing the scores for all possible pairings
        (n x n) matrix of floats

    Returns
    -------
    alloc: The matrix form pairing allocations
        (n x n) matrix of integers
    pairs: The list form of pairing allocations
        (n / 2 x (2, 2, 1)) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id), pod]
    """
    n_players = len(standings)
    ix = {pos+start: player_id for pos, player_id in enumerate(standings[:,PID])}
    pods = {player[PID]: player[POD] for player in standings}

    # Construct initial draw
    if alloc is None:
        alloc = np.zeros((n_players, n_players), dtype=int)
        prob = scores - scores.max(axis=1, keepdims=True)
        prob = np.exp(prob.astype(np.float32)/4)
        for _ in range(int(n_players / 2)):
            mask = (1 - alloc.sum(axis=1))
            # Choose Player A
            a = mask.argmax()
            # Choose Player B
            p = prob[a]
            p = np.maximum(p * mask, 0)
            b = np.random.choice(np.arange(n_players)[p==p.max()])

            alloc[a, b] = 1
            alloc[b, a] = 1

    # Run one-up-one-down to iteratively improve draw
    score_q = [(alloc*scores).sum()/n_players]

    brackets = [n for n in range(n_players) if (standings[n,[WINS]] != standings[n-1,[WINS]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = Points.valid  - 2 * n_odd_brackets * (Points.valid + Points.overall_win_distance) / n_players

    while max(score_q) < max_score:
        # If the worst score is in the first half of the draw,  run through first to last
        # If the worst score is in the second half of the draw, run through last to first
        isFwdSort = scores[alloc==1].argmin() / len(scores) < 0.5
        isFwdSort = (-1)**(1+isFwdSort)

        for a in range(n_players-1)[::isFwdSort]:
            b = alloc[a].argmax()
            if scores[a,b] == scores[a].max():
                continue

            # Search one-up-one-down
            # Creates series c = [a+1, a-1, a+2, a-2...]
            for i in range(2, 2 * n_players):
                # Select c and d
                c = a + (i // 2) * (-1)**i
                if c not in range(n_players-1):
                    continue
                d = alloc[c].argmax()

                opts = [scores[a,b]+scores[c,d],
                        scores[a,c]+scores[b,d],
                        scores[a,d]+scores[b,c]]
                # Swap pairs if there is a better allocation
                if max(opts[1], opts[2]) > opts[0]:
                    opt = np.argmax(opts)

                    if opt == 1: alloc = swap_pairs((a,b), (c,d), alloc)
                    if opt == 2: alloc = swap_pairs((a,b), (d,c), alloc)
                    break

        score = (alloc*scores).sum()/n_players
        score_q.append(score)
        if score_q[-1] == score_q[-2]:
            break

    #if len(score_q) > 1: print(["%.2f" % v for v in score_q])

    # Create list form of pairings
    pairs = []
    for a in range(n_players):
        b = alloc[a].argmax()
        if a < b:
            a, b = a + start, b + start
            pairs.append([(a, ix[a]), (b, ix[b]), pods[ix[a]]])

    return alloc, pairs

def swap_pairs(pair_1, pair_2, alloc):
    """
    Helper function to swap players between two pairings
    Original pairing of (a,b) and (c,d) becomes (a,c) and (b,d)

    Parameters
    ----------
    pair_1: Tuple containing first pair to swap (a,b)
    pair_2: Tuple containing first pair to swap (c,d)
    alloc: The matrix form pairing allocations
        (n x n) matrix of integers

    Returns
    -------
    alloc: The matrix form pairing allocations with [(a,b), (c,d)] -> [(a,c), (b,d)]
        (n x n) matrix of integers
    """
    a, b = pair_1
    c, d = pair_2

    alloc[a, b] = 0
    alloc[b, a] = 0
    alloc[c, d] = 0
    alloc[d, c] = 0
    alloc[a, c] = 1
    alloc[c, a] = 1
    alloc[b, d] = 1
    alloc[d, b] = 1

    return alloc

def get_splits(standings):
    """
    Helper method to find the points at which the brackets can be split with even numbers of players

    Parameters
    ----------
    standings: List of standings for all players still in the tournament
        (n x 4) list containing [player_id, player_wins, pod_id, player_wins_in_pod]

    Returns
    -------
    splits: List of indices where brackets can be split
        (b x 0) list of integers [breakpoint1, breakpoint2, ..., breakpointb]
    """
    n_players = len(standings)
    n_pods    = len(set(standings[:,POD]))

    # Sort standings by bracket and player wins
    standings = sort(standings, [TBRK, WINS, POD], [1, -1, 1])

    # Calculate splits
    if n_pods > 1:
        # If players are allocated to pods
        # Add split if there is a break across pods
        splits = [n for n in range(n_players) if (standings[n,[POD]] != standings[n-1,[POD]]).any()] + [n_players]
    else:
        # If players are not allocated to pods
        # Add split if more than 40 players are on the same overall points
        splits, breakpoints = [0], [0]
        breakpoints += [n for n in range(1, n_players) if (standings[n,[WINS]] != standings[n-1,[WINS]]).any()] + [n_players]
        for break_1, break_2 in zip(breakpoints[:-1], breakpoints[1:]):
            if break_2 - break_1 > 40:
                splits += list(range(break_1 + 40 + break_1 % 2, break_2 - 40, 40))
        splits.append(n_players)

    return splits

def sort(x, cols, orders, preshuffle=False):
    """
    Helper method to sort an array by a combination of columns in forward or reverse orders

    Parameters
    ----------
    x: Array to sort
        (n x m) numpy array
    cols: List of the columns to sort by
        list [(0-m), ..., (0-m)]
    orders: List of the order to to sort columns by. 1 == ascending, -1 == descending
        list [(1,-1), ..., (1,-1)]
    preshuffle: Flag whether to shuffle before sorting
        boolean

    Returns
    -------
    x: Sorted array
        (n x m) numpy array
    """
    assert(len(cols) == len(orders))

    if preshuffle:
        x = x.copy()
        np.random.shuffle(x)

    for col, order in zip(cols, orders):
        ind = np.argsort(x[:,col], kind='stable')
        x   = x[ind][::order]

    return x

## Draw Printing Methods

In [14]:
standings

array([['7546101', 5, 1.0, 1, 0],
       ['3731519', 4, 0.8392969240426867, 1, 0],
       ['1942928', 4, 0.7991211550533585, 1, 0],
       ['6830055', 4, 0.7890772128060264, 1, 0],
       ['8087259', 3, 0.6384180790960452, 1, 0],
       ['7772032', 3, 0.6258631512868801, 1, 0],
       ['6868613', 3, 0.5856873822975518, 1, 0],
       ['7425364', 2, 0.4274952919020716, 1, 0],
       ['6482714', 4, 0.7865662272441933, 2, 0],
       ['5530784', 4, 0.7865662272441933, 2, 0],
       ['1078170', 3, 0.6283741368487131, 2, 0],
       ['1804471', 3, 0.6258631512868801, 2, 0],
       ['4627023', 3, 0.5856873822975518, 2, 0],
       ['8091524', 3, 0.5850596359070935, 2, 0],
       ['4154883', 2, 0.4274952919020716, 2, 0],
       ['7043775', 2, 0.42498430634023854, 2, 0],
       ['3329087', 3, 0.5750156936597615, 3, 0],
       ['1201457', 3, 0.5725047080979284, 3, 0],
       ['5922289', 2, 0.4143126177024482, 3, 0],
       ['6667846', 2, 0.4143126177024482, 3, 0],
       ['9097730', 2, 0.3747645951

In [15]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([player[0], id_to_name[player[0]], player[1], rank+1, player[3], player[2]])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Standing', 'Pod', 'Rank']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        b, a, pod = pair[0], pair[1], pair[2]
        score = scores[b[0], a[0]]
        row = [table_num+1, pod, id_to_name[a[1]], a[1], int(a[0])+1, id_to_name[b[1]], b[1], int(b[0])+1, score]
        table.append(row)
    headers = ['Table', 'Pod', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs):
    byes = []
    for pairs in pairs_h:
        for pair in pairs:
            a, b = pair[0][1], pair[1][1]
            if min(a, b) < 0:
                byes += [max(a, -1), max(b, -1)]
    byes = Counter(np.reshape(byes, -1))

    print('----- BYES -----')
    for k, v in byes.items():
        if k < 0:
            print('Byes', ': ', v)
        else:
            print(k, ': ', v)

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes [Done]
- Allocating pods [Done]
- Iterative sampling methods to improve draw quality [Done]
- Balancing tail end pod sizes
- Variable size pods
- etc

In [11]:
NUMBER_OF_PLAYERS = 31
ROUNDS  = ['swiss', 'swiss', 'swiss',
           'pod1', 'pod1', 'pod1',
           'pod2', 'pod2', 'pod2',
           'swiss', 'swiss', 'swiss']
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.0001

In [12]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [16]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []
number_of_players = NUMBER_OF_PLAYERS + NUMBER_OF_PLAYERS % 2

for rnd in range(NUMBER_OF_ROUNDS):
    standings, splits, scores, alloc, pairs = create_draw(players, results, ROUNDS, mode='test')
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / number_of_players)
    print('Maximum GOF:', 16 - 15 * (np.array(splits) % 2).sum() * 2 / number_of_players)
    print("Frequencies: " + ', '.join([str(int(k)) + "pts: " + str(int(v / 2)) 
                                 for k, v in Counter((scores * alloc).reshape(-1)).items() if k != 0]) )
    print_standings(standings, id_to_name, )
    print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, rnd, p_draw=PROBABILITY_OF_DRAW)
    results += result

print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + "pts: " + str(int(v / 2))
   for k, v in Counter((np.array(scores_h)*np.array(alloc_h))[-NUMBER_OF_ROUNDS:].reshape(-1)).items() if k != 0]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 16.0
Maximum GOF: 16.0
Frequencies: 16pts: 16

 ID       Name                 Wins    Standing    Pod    Rank
-------  -----------------  ------  ----------  -----  ------
9097730  Maria Sullivan          0           1      1       0
5528377  Arthur Reyes            0           2      1       0
3336349  Scott Bell              0           3      1       0
7772032  Kelly Wu                0           4      1       0
4032830  Brandon White           0           5      1       0
6868613  Ashley Hawkins          0           6      1       0
6221163  David Ward              0           7      1       0
1078170  Jeffrey Hernandez       0           8      1       0
1942928  Erica Evans             0           9      1       0
1804471  Mary Cook               0          10      1       0
4154883  Julie Gonzales          0          11      1       0
7425364  Daniel Montgomery       0          12      1       0
7546101  Joyce Finley            0          13 

----- Round 6 -----
Goodness of Fit: 14.75
Maximum GOF: 16.0
Frequencies: 8pts: 2, 16pts: 12, 14pts: 2

 ID       Name                 Wins    Standing    Pod      Rank
-------  -----------------  ------  ----------  -----  --------
9097730  Maria Sullivan          5           1      1  1
4032830  Brandon White           4           2      1  0.839297
1936805  Matthew Townsend        4           3      1  0.799121
5638781  Kimberly Poole          4           4      1  0.785938
7772032  Kelly Wu                3           5      1  0.638418
3329087  James Nunez             3           6      1  0.628374
5530784  Kimberly Smith          3           7      1  0.588198
1078170  Jeffrey Hernandez       2           8      1  0.424357
7043775  Ralph Chandler          4           9      2  0.789077
6830055  Brad Caldwell           4          10      2  0.786566
5922289  Antonio Wilson          3          11      2  0.625863
7672887  Todd Simpson            3          12      2  0.625863
552837

TypeError: '<' not supported between instances of 'str' and 'int'